In [ ]:
%pip install qdrant-client

In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import numpy as np

from iris.config.data_pipeline_config_manager import DataPipelineConfigManager

In [2]:
data_config = DataPipelineConfigManager()
qdrant_config = data_config.qdrant_config

In [ ]:
# Connect to Qdrant Cloud
client = QdrantClient(
    url=qdrant_config.url,
    api_key=qdrant_config.api_key
)

# Test connection
try:
    collections = client.get_collections()
    print("Successfully connected to Qdrant Cloud!")
    print("Available collections:", collections)
except Exception as e:
    print("Connection failed:", str(e))

Successfully connected to Qdrant Cloud!
Available collections: collections=[]


In [6]:
# Create a test collection
collection_name = "test_collection"
vector_size = 4  # Using small vectors for demonstration

# Create collection if it doesn't exist
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

C:\Users\ice\AppData\Local\Temp\ipykernel_22828\1141237782.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [7]:
# Create some sample points
vectors = [
    np.random.rand(vector_size) for _ in range(3)
]
payloads = [
    {"description": "First point"},
    {"description": "Second point"},
    {"description": "Third point"}
]

# Upload points
points = [
    PointStruct(
        id=idx,
        vector=vector.tolist(),
        payload=payload
    )
    for idx, (vector, payload) in enumerate(zip(vectors, payloads))
]

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
# Perform a search
search_vector = np.random.rand(vector_size)
search_results = client.search(
    collection_name=collection_name,
    query_vector=search_vector,
    limit=2
)

print("Search results:")
for result in search_results:
    print(f"Point {result.id}: score={result.score}, payload={result.payload}")

Search results:
Point 1: score=0.8209628, payload={'description': 'Second point'}
Point 0: score=0.6575983, payload={'description': 'First point'}


C:\Users\ice\AppData\Local\Temp\ipykernel_22828\992534649.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(


In [12]:
# Fetch a specific vector by ID
point_id = 1  # Get the second point we inserted
retrieved_point = client.retrieve(
    collection_name=collection_name,
    ids=[point_id],
    with_vectors=True  # Include the actual vector data
)

if retrieved_point:
    point = retrieved_point[0]
    print(f"Retrieved point {point_id}:")
    print(f"Vector: {point.vector}")
    print(f"Payload: {point.payload}")
else:
    print(f"No point found with ID {point_id}")

Retrieved point 1:
Vector: [0.4674162, 0.40356088, 0.46604568, 0.6336104]
Payload: {'description': 'Second point'}


# Accessing Qdrant Cloud GUI
You can view and interact with your vectors through the Qdrant Cloud Console:
1. Go to https://cloud.qdrant.io/
2. Login with your credentials
3. Select your cluster
4. Navigate to the "Collections" tab to see your `test_collection`

In [11]:
# Print direct URL to Qdrant dashboard
dashboard_url = f"{qdrant_config.url}/dashboard"
print(f"View your vectors in Qdrant Dashboard:\n{dashboard_url}")

# For reference, also print the full connection URL
print(f"\nCurrent connection URL:\n{qdrant_config.url}")

View your vectors in Qdrant Dashboard:
https://faa8f550-ff00-4890-b04e-1151b28d57a4.eu-west-2-0.aws.cloud.qdrant.io:6333/dashboard

Current connection URL:
https://faa8f550-ff00-4890-b04e-1151b28d57a4.eu-west-2-0.aws.cloud.qdrant.io:6333
